# Winamax Tennis

In [1]:
#pip install selenium
#pip install webdriver-manager
#pip install BeautifulSoup4

In [9]:
import pandas as pd
import numpy as np
import re
import requests
import webbrowser
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from datetime import datetime, date, timedelta
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
driver = webdriver.Chrome("C:/Users/Valentin/Documents/Winamax/chromedriver")

In [10]:
############ Parameters
sport = "tennis"
history = 15 #days
nblastmatch = [3,6,10] #days
weight = [3,2,1] #days
nblines = 9999 #rows
threshold = 1.5
OddMini = 1.15
############

Day = "Today"
# Day = "Tomorrow"
special_match = "https://www.oddsportal.com/search/Berrettini%20M.%20/"
tournaments = ["https://www.oddsportal.com/tennis/united-kingdom/wta-wimbledon/",
               "https://www.oddsportal.com/tennis/united-kingdom/atp-wimbledon/"]

MODE = "tournaments" #tournaments #match #day

In [11]:
if MODE == "day":
    if Day == "Today":
        day = datetime.today()
    else:
        day = datetime.today() + timedelta(days = 1) 
    driver.get('https://www.oddsportal.com/matches/tennis/'+ day.strftime("%Y%m%d") + '/')
    pages="1"
else:
    if MODE == "match":
        driver.get(special_match)
        pages="1"
    else:
        if MODE == "tournaments":
            pages=tournaments

d = pd.DataFrame()
for page in pages:
    if MODE == "tournaments":
        driver.get(page)
        
    soup = BeautifulSoup(driver.page_source)
    p = soup.find_all('strong')

    ############ Creation of the dataset
    T = soup.find('table', class_='table-main')
    my_table = T
    T.find('tr',class_='dark center').decompose()

    table_data = []
    for row in my_table.findAll('tr'):
        row_data = []    

        for cell in row.findAll('td'):
            row_data.append(cell.text)
        if(len(row_data) > 0):
            data_item = {"DateTime": row_data[0],
                          "Players": row_data[1],
                          "Odd1": row_data[2],
                          "Odd2": row_data[3],
                          "Add": row_data[4]
                          #"Bs": row_data[5]
            }
            table_data.append(data_item)
    data = pd.DataFrame(table_data)
#     print(str(re.search('.*/(.*)/', page)))
    data["tournament"] = re.search('.*/(.*)/', str(page)).group(1)
    d = pd.concat([d, data.head(nblines)])
    ############ Dataset
df = d.reindex()

df = df[df['Players'].str.count('/')<2] #Remove duo
df = df[df['Odd1'].str.contains("[1-9]")]
df = df[df['Odd2'].str.contains("[1-9]")]
# df = df[df['Players'].str.count('-')==1] # Remove names with '-'
df = df[np.invert(df['Odd1'].str.contains(":"))] # Remove match already started
df = df.assign(Player1 = df['Players'].map(lambda x: x.split(' - ')[0]),
               Player2 = df['Players'].map(lambda x: x.split(' - ')[1])).drop("Add", axis = 1).head(nblines)
Players1 = df['Player1'].tolist()
Players2 = df['Player2'].tolist()
Players1 = [w.replace('\xa0', '') for w in Players1]
Players2 = [w.replace('\xa0', '') for w in Players2]
Players = [None]*(len(Players1)+len(Players2))
Players[::2] = Players1
Players[1::2] = Players2
Odd1 = df['Odd1'].tolist()
Odd2 = df['Odd2'].tolist()
Odd1 = [w.replace('\xa0', '') for w in Odd1]
Odd2 = [w.replace('\xa0', '') for w in Odd2]
Odds = [None]*(len(Odd1)+len(Odd2))
Odds[::2] = Odd1
Odds[1::2] = Odd2
params = [Players,Odds]
params[0]
Players = [w.strip().replace(' ', '+') for w in Players]
#Players
#     data_res = pd.concat([data_res, df.head(nblines).reindex()])
data_res = df#.reindex()
data_res.tail()

,DateTime,Players,Odd1,Odd2,tournament,Player1,Player2
4,12:00,Federer R. - Sonego L.,1.27,3.97,atp-wimbledon,Federer R.,Sonego L.
5,12:00,Fucsovics M. - Rublev A.,5.29,1.17,atp-wimbledon,Fucsovics M.,Rublev A.
6,12:00,Hurkacz H. - Medvedev D.,6.47,1.12,atp-wimbledon,Hurkacz H.,Medvedev D.
7,12:00,Khachanov K. - Korda S.,1.88,1.95,atp-wimbledon,Khachanov K.,Korda S.
8,12:00,Shapovalov D. - Bautista-Agut R.,1.69,2.21,atp-wimbledon,Shapovalov D.,Bautista-Agut R.


In [12]:
i=1

############Loop on Players
for Player in Players:
    PlayerName = Player
    Player = Player.strip().replace(" ", "+")
    p = Player.strip() + '/' + sport + '/'
    driver.get('https://www.oddsportal.com/search/results/' + p)
    soup = BeautifulSoup(driver.page_source)
    p = soup.find_all('strong')
    
    ############ Getting URL
    paragraphs = []
    for x in p:
        paragraphs.append(str(x))

    if any("(" in s for s in paragraphs):
        result = [s for s in paragraphs if "(" in s]
        result = [re.sub(".*\(", "", x) for x in result]
        result = [re.sub("\).*", "", x) for x in result]
        if result == ['0']:
            PlayerCode = soup.find('tr', class_='odd').find('a').get('href').replace("/search/results/","").replace("/","")
            #page = getPage(soup.find('tr', class_='odd').find('a').get('href').split("[/:]").replace("/", "") + '/')
            driver.get('https://www.oddsportal.com/search/results/' + PlayerCode + '/')
            soup = BeautifulSoup(driver.page_source)

    ############ Creation of the dataset
    T = soup.find('table', class_='table-main')
    if (T.find('tr',class_='dark center')!=None):
        my_table = T
        T.find('tr',class_='dark center').decompose()

        table_data = []
        for row in my_table.findAll('tr'):
            row_data = []    

            for cell in row.findAll('td'):
                row_data.append(cell.text)
            if(len(row_data) > 0):
                data_item = {"DateTime": row_data[0],
                             "Players": row_data[1],
                             "Result": row_data[2],
                             "Odd1": row_data[3],
                             "Odd2": row_data[4],
                             "Bs": row_data[5]
                }
                table_data.append(data_item)
                
        data = pd.DataFrame(table_data)
        #data["Date"] = data["DateTime"].astype(str).str[0:5]
        #data = data[data["Date"] > (day + timedelta(days = -history)).strftime("%d/%m")]
        #if (len(data.index) != 0):
        ############ Dataset
        def cancel(row):
            if row['Result'].find(":")!= -1 :
                 return 0
            else :
                 return 1

        df = data
        df["cancel"] = df.apply(lambda row: cancel(row), axis=1)
        df = df[df["cancel"] == 0]
        df = df.assign(Player1 = df['Players'].map(lambda x: x.split('-')[0]),
                       Player2 = df['Players'].map(lambda x: x.split('-')[1]),
                       Score1 = df['Result'].map(lambda x: x.split(':')[0]),
                       Score2 = df['Result'].map(lambda x: x.split(':')[1])
                      ).drop("Players", axis = 1)

        
        df = df[df['Odd1'].str.contains("[1-9]")]
        df = df[df['Odd2'].str.contains("[1-9]")]
        
        ############ Data column type
        df['Score1'] = df['Score1'].astype(int, errors = 'raise')
        df['Score2'] = df['Score2'].astype(int, errors = 'raise')
        df['Odd1'] = df['Odd1'].astype(float, errors = 'raise')
        df['Odd2'] = df['Odd2'].astype(float, errors = 'raise')
        df['Player1'] = df['Player1'].astype(str)
        df['Player2'] = df['Player2'].astype(str)
        
        #print(Player.str())
        #PlayerName = df[df["Player2"].str.contains(PlayerName)]["Player2"].values[0]
        
        ############ Functions
        def Winner (row):
           if (row['Score1'] > row['Score2']) :
              return row['Player1']
           else :
              return row['Player2']

        def OddWinner (row):
           if (row['Score1'] > row['Score2']) :
              return row['Odd1']
           else :
              return row['Odd2']

        def round_of_rating(number):
            if number < 3 :
                if number > 0.5 :
                    return number
                else:
                    return 0.5
            else :
                return 3

        def Challenger (row):
            if (row['Odd1'] < row['Odd2']) :
                return  row['Player1']
            else :
                return  row['Player2']

        def NbSetChallenger (row):
            if (row['Challenger'] == row['Player1']) :
                return  row['Score1']/(row['Score1']+row['Score2']) #2/3 or 1
            else :
                return  row['Score2']/(row['Score1']+row['Score2']) #2/3 or 1
            
        def Analysis (row):
            if (row['TargetPlayer'] in row['Winner'].strip()) :
                if (row['Challenger'].strip() == row['TargetPlayer']) :
                    return  11 + 1/round_of_rating(row['OddDiff']) - 3*(1-row['NbSetChallenger'])
                else:
                    return  13 + round_of_rating(row['OddDiff']) - 3*(1-row['NbSetChallenger'])
            else :
                if (row['Challenger'].strip() == row['TargetPlayer']) :
                    return  10-3 - round_of_rating(row['OddDiff']) + 3*(1-row['NbSetChallenger'])
                else:
                    return  10-1 - 1/round_of_rating(row['OddDiff']) + 3*(1-row['NbSetChallenger'])


        def toBet(row, lastMatchs):
            if (row["Analysis1_" + str(lastMatchs)] > row["Analysis2_" + str(lastMatchs)]):
                if (row["Odd1"] > row["Odd2"]):
                    return row["Analysis1_" + str(lastMatchs)] - row["Analysis2_" + str(lastMatchs)]
                else:
                    return -(row["Analysis1_" + str(lastMatchs)] - row["Analysis2_" + str(lastMatchs)])
            else:
                if (row["Odd2"] > row["Odd1"]):
                    return row["Analysis2_" + str(lastMatchs)] - row["Analysis1_" + str(lastMatchs)]
                else:
                    return -(row["Analysis2_" + str(lastMatchs)] - row["Analysis1_" + str(lastMatchs)])


        ############ Aplply Functions   
        df["TargetPlayer"] = Player.replace("+", " ").strip()
        df["Winner"] = df.apply(lambda row: Winner(row), axis=1)
        df["OddWinner"] = df.apply(lambda row: OddWinner(row), axis=1)
        df["OddDiff"] = abs(df['Odd1'] - df['Odd2'])
        df["Challenger"] = df.apply(lambda row: Challenger(row), axis=1)
        df["NbSetChallenger"] = df.apply(lambda row: NbSetChallenger(row), axis=1)
        dfall = df
        
        for nmatch in nblastmatch:
            df = dfall.head(nmatch)
            if (i % 2 == 0):
                data_res.loc[data_res['Players'].str.contains(Player.replace("+", " ").strip(), na=False), 'Analysis2_' + str(nmatch)] = round(df.apply(lambda row: Analysis(row), axis=1).mean(),2)
                data_res["toBet_" + str(nmatch)] = data_res.apply(lambda row: toBet(row, nmatch), axis=1)   
            else:
                data_res.loc[data_res['Players'].str.contains(Player.replace("+", " ").strip(), na=False), 'Analysis1_' + str(nmatch)] = round(df.apply(lambda row: Analysis(row), axis=1).mean(),2)
      
            ##Nb Win when favorite, Nb Loss when Favorite .....

    i=i+1

data_res.head(5)

,DateTime,Players,Odd1,Odd2,tournament,Player1,Player2,Analysis1_3,Analysis1_6,Analysis1_10,Analysis2_3,toBet_3,Analysis2_6,toBet_6,Analysis2_10,toBet_10
1,10:00,Badosa G. P. - Muchova K.,2.35,1.62,wta-wimbledon,Badosa G. P.,Muchova K.,11.39,10.80,10.31,12.12,-0.73,10.33,0.47,10.46,-0.15
2,10:00,Barty A. - Krejcikova B.,1.35,3.34,wta-wimbledon,Barty A.,Krejcikova B.,11.00,11.02,10.98,11.52,0.52,11.53,0.51,11.55,0.57
3,10:00,Gauff C. - Kerber A.,1.77,2.09,wta-wimbledon,Gauff C.,Kerber A.,13.80,12.01,11.63,10.68,-3.12,11.24,-0.77,10.95,-0.68
4,10:00,Keys M. - Golubic V.,1.50,2.66,wta-wimbledon,Keys M.,Golubic V.,11.91,11.30,10.50,11.26,-0.65,11.30,0.00,11.43,0.93
5,10:00,Pliskova Ka. - Samsonova L.,1.91,1.92,wta-wimbledon,Pliskova Ka.,Samsonova L.,11.66,10.04,10.18,11.58,-0.08,11.73,1.69,11.84,1.66


In [14]:
data_res

,DateTime,Players,Odd1,Odd2,tournament,Player1,Player2,Analysis1_3,Analysis1_6,Analysis1_10,Analysis2_3,toBet_3,Analysis2_6,toBet_6,Analysis2_10,toBet_10
1,10:00,Badosa G. P. - Muchova K.,2.35,1.62,wta-wimbledon,Badosa G. P.,Muchova K.,11.39,10.80,10.31,12.12,-0.73,10.33,0.47,10.46,-0.15
2,10:00,Barty A. - Krejcikova B.,1.35,3.34,wta-wimbledon,Barty A.,Krejcikova B.,11.00,11.02,10.98,11.52,0.52,11.53,0.51,11.55,0.57
3,10:00,Gauff C. - Kerber A.,1.77,2.09,wta-wimbledon,Gauff C.,Kerber A.,13.80,12.01,11.63,10.68,-3.12,11.24,-0.77,10.95,-0.68
4,10:00,Keys M. - Golubic V.,1.50,2.66,wta-wimbledon,Keys M.,Golubic V.,11.91,11.30,10.50,11.26,-0.65,11.30,0.00,11.43,0.93
5,10:00,Pliskova Ka. - Samsonova L.,1.91,1.92,wta-wimbledon,Pliskova Ka.,Samsonova L.,11.66,10.04,10.18,11.58,-0.08,11.73,1.69,11.84,1.66
6,10:00,Raducanu E. - Tomljanovic A.,2.13,1.74,wta-wimbledon,Raducanu E.,Tomljanovic A.,11.24,10.65,10.40,11.39,-0.15,11.37,-0.72,11.12,-0.72
7,10:00,Rybakina E. - Sabalenka A.,2.55,1.54,wta-wimbledon,Rybakina E.,Sabalenka A.,10.10,10.33,10.16,11.00,-0.90,10.28,0.05,10.86,-0.70
8,10:00,Swiatek I. - Jabeur O.,1.73,2.15,wta-wimbledon,Swiatek I.,Jabeur O.,11.44,11.10,10.23,11.51,0.07,10.41,-0.69,10.85,0.62
1,12:00,Auger-Aliassime F. - Zverev A.,3.38,1.33,atp-wimbledon,Auger-Aliassime F.,Zverev A.,8.42,8.80,8.94,11.08,-2.66,10.10,-1.30,10.59,-1.65
2,12:00,Berrettini M. - Ivashka I.,1.14,6.04,atp-wimbledon,Berrettini M.,Ivashka I.,11.08,11.16,10.95,12.04,0.96,11.43,0.27,10.99,0.04


In [5]:
varAnalysis = ["DateTime", "Players"]
toBet = []
data_res["toBet"] = 0
i = 0
for nmatch in nblastmatch:
    varAnalysis.append("Analysis1_"+ str(nmatch))
    varAnalysis.append("Analysis2_"+ str(nmatch))
    toBet.append("toBet_"+ str(nmatch))
    data_res["toBet_"+ str(nmatch)] = data_res["toBet_"+ str(nmatch)].astype(float, errors = 'raise')
    data_res["toBet"] = data_res["toBet"] + weight[i]*data_res["toBet_"+ str(nmatch)]
    i = i+1
varAnalysis
res = data_res
res.drop(varAnalysis, axis=1, inplace=True)
res['Odd1'] = res['Odd1'].astype(float, errors = 'raise')
res['Odd2'] = res['Odd2'].astype(float, errors = 'raise')
# res = res.sort_values(by = toBet, ascending = False, axis=0))
res = res.sort_values(by = "toBet", ascending = False, axis=0)
# res2 = res[(np.array(res["toBet"]>10) | np.array(res["toBet"]<-10)) & (np.array(res["Odd1"]>OddMini) & np.array(res["Odd2"]>OddMini))]
res2 = res[(np.array(res["toBet"]>10) | np.array(res["toBet"]<-10))]
res2.reset_index(drop=True).style.background_gradient(cmap="Blues_r").set_precision(2)

,Odd1,Odd2,tournament,Player1,Player2,toBet_3,toBet_6,toBet_10,toBet
0,1.78,2.08,wta-wimbledon,Gauff C.,Kerber A.,-3.12,-0.77,-0.68,-11.58
1,3.39,1.33,atp-wimbledon,Auger-Aliassime F.,Zverev A.,-2.66,-1.30,-1.65,-12.23


,Odd1,Odd2,tournament,Player1,Player2,toBet_3,toBet_6,toBet_10,toBet
6,6.43,1.13,atp-wimbledon,Hurkacz H.,Medvedev D.,1.74,0.61,0.21,6.65
7,1.88,1.96,atp-wimbledon,Khachanov K.,Korda S.,0.29,0.91,0.98,3.67
2,1.14,6.05,atp-wimbledon,Berrettini M.,Ivashka I.,0.96,0.27,0.04,3.46
2,1.35,3.34,wta-wimbledon,Barty A.,Krejcikova B.,0.52,0.51,0.57,3.15
4,1.27,3.96,atp-wimbledon,Federer R.,Sonego L.,0.37,-0.30,-0.08,0.43
8,1.73,2.15,wta-wimbledon,Swiatek I.,Jabeur O.,0.07,-0.69,0.62,-0.55
4,1.50,2.66,wta-wimbledon,Keys M.,Golubic V.,-0.65,0.00,0.93,-1.02
1,2.35,1.62,wta-wimbledon,Badosa G. P.,Muchova K.,-0.73,0.47,-0.15,-1.40
5,5.27,1.17,atp-wimbledon,Fucsovics M.,Rublev A.,-0.36,0.15,-0.79,-1.57
6,2.13,1.74,wta-wimbledon,Raducanu E.,Tomljanovic A.,-0.15,-0.72,-0.72,-2.61


i=1
data_res["Analysis1"] = np.nan
data_res["Analysis2"] = np.nan
############Loop on Players
for Player in Players:
    PlayerName = Player
    Player = Player.strip().replace(" ", "+")
    p = Player.strip() + '/' + sport + '/'
    driver.get('https://www.oddsportal.com/search/results/' + p)
    soup = BeautifulSoup(driver.page_source)
    p = soup.find_all('strong')
    
    ############ Getting URL
    paragraphs = []
    for x in p:
        paragraphs.append(str(x))

    if any("(" in s for s in paragraphs):
        result = [s for s in paragraphs if "(" in s]
        result = [re.sub(".*\(", "", x) for x in result]
        result = [re.sub("\).*", "", x) for x in result]
        if result == ['0']:
            PlayerCode = soup.find('tr', class_='odd').find('a').get('href').replace("/search/results/","").replace("/","")
            #page = getPage(soup.find('tr', class_='odd').find('a').get('href').split("[/:]").replace("/", "") + '/')
            driver.get('https://www.oddsportal.com/search/results/' + PlayerCode + '/')
            soup = BeautifulSoup(driver.page_source)

    ############ Creation of the dataset
    T = soup.find('table', class_='table-main')
    if (T.find('tr',class_='dark center')!=None):
        my_table = T
        T.find('tr',class_='dark center').decompose()

        table_data = []
        for row in my_table.findAll('tr'):
            row_data = []    

            for cell in row.findAll('td'):
                row_data.append(cell.text)
            if(len(row_data) > 0):
                data_item = {"DateTime": row_data[0],
                             "Players": row_data[1],
                             "Result": row_data[2],
                             "Odd1": row_data[3],
                             "Odd2": row_data[4],
                             "Bs": row_data[5]
                }
                table_data.append(data_item)
        data = pd.DataFrame(table_data)
        #data["Date"] = data["DateTime"].astype(str).str[0:5]
        #data = data[data["Date"] > (day + timedelta(days = -history)).strftime("%d/%m")]
        #if (len(data.index) != 0):
        ############ Dataset
        def cancel(row):
            if row['Result'].find(":")!= -1 :
                 return 0
            else :
                 return 1

        df = data
        df["cancel"] = df.apply(lambda row: cancel(row), axis=1)
        df = df[df["cancel"] == 0]
        df = df.assign(Player1 = df['Players'].map(lambda x: x.split('-')[0]),
                       Player2 = df['Players'].map(lambda x: x.split('-')[1]),
                       Score1 = df['Result'].map(lambda x: x.split(':')[0]),
                       Score2 = df['Result'].map(lambda x: x.split(':')[1])
                      ).drop("Players", axis = 1)

        
        df = df[df['Odd1'].str.contains("[1-9]")]
        df = df[df['Odd2'].str.contains("[1-9]")]
        
        ############ Data column type
        df['Score1'] = df['Score1'].astype(int, errors = 'raise')
        df['Score2'] = df['Score2'].astype(int, errors = 'raise')
        df['Odd1'] = df['Odd1'].astype(float, errors = 'raise')
        df['Odd2'] = df['Odd2'].astype(float, errors = 'raise')
        df['Player1'] = df['Player1'].astype(str)
        df['Player2'] = df['Player2'].astype(str)
        
        #print(Player.str())
        #PlayerName = df[df["Player2"].str.contains(PlayerName)]["Player2"].values[0]
        df = df.head(nblastmatch[-1])
        
        ############ Functions
        def Winner (row):
           if (row['Score1'] > row['Score2']) :
              return row['Player1']
           else :
              return row['Player2']

        def OddWinner (row):
           if (row['Score1'] > row['Score2']) :
              return row['Odd1']
           else :
              return row['Odd2']

        def round_of_rating(number):
            if number < 3 :
                if number > 0.5 :
                    return number
                else:
                    return 0.5
            else :
                return 3

        def Challenger (row):
            if (row['Odd1'] < row['Odd2']) :
                return  row['Player1']
            else :
                return  row['Player2']

        def NbSetChallenger (row):
            if (row['Challenger'] == row['Player1']) :
                return  row['Score1']/(row['Score1']+row['Score2']) #2/3 or 1
            else :
                return  row['Score2']/(row['Score1']+row['Score2']) #2/3 or 1
            
        def Analysis (row):
            if (row['TargetPlayer'] in row['Winner'].strip()) :
                if (row['Challenger'].strip() == row['TargetPlayer']) :
                    return  1 + 1/round_of_rating(row['OddDiff']) - 3*(1-row['NbSetChallenger'])
                else:
                    return  3 + round_of_rating(row['OddDiff']) - 3*(1-row['NbSetChallenger'])
            else :
                if (row['Challenger'].strip() == row['TargetPlayer']) :
                    return  -3 - round_of_rating(row['OddDiff']) + 3*(1-row['NbSetChallenger'])
                else:
                    return  -1 - 1/round_of_rating(row['OddDiff']) + 3*(1-row['NbSetChallenger'])
            
        ############ Aplply Functions
        df["TargetPlayer"] = Player.replace("+", " ").strip()
        df["Winner"] = df.apply(lambda row: Winner(row), axis=1)
        df["OddWinner"] = df.apply(lambda row: OddWinner(row), axis=1)
        df["OddDiff"] = abs(df['Odd1'] - df['Odd2'])
        df["Challenger"] = df.apply(lambda row: Challenger(row), axis=1)
        df["NbSetChallenger"] = df.apply(lambda row: NbSetChallenger(row), axis=1)
        df["Analysis"] = df.apply(lambda row: Analysis(row), axis=1)
        
        ############ Results
#         for lastMatchs in nblastmatch:
#             if (i % 2 == 0) :
#                 data_res.at[data_res['Players'].str.contains(Player.replace("+", " ").strip()), 'Analysis2_' + str(lastMatchs)] = round(df["Analysis"].mean(),2)
#             else:
#                 data_res.at[data_res['Players'].str.contains(Player.replace("+", " ").strip()), 'Analysis1_' + str(lastMatchs)] = round(df["Analysis"].mean(),2)
        
#             res = data_res        
            
#             ##Nb Win when favorite, Nb Loss when Favorite .....
#             ##Ecart de cote for each case
#             ## Nb sets win when loss favorite/not ... etc

#             def toBet(row):
#                 # BUG ICI
#                 if (abs(row["Analysis1_" + str(lastMatchs)] - row["Analysis2_" + str(lastMatchs)])>threshold):
#                     if (row["Analysis1_" + str(lastMatchs)] > row["Analysis2_" + str(lastMatchs)]):
#                         if (row["Odd1"] > row["Odd2"]):
#                             return 10
#                     else:
#                         if (row["Odd2"] > row["Odd1"]):
#                             return 10
#                     return -10
#                 else:
#                     return 0
#             res["toBet_" + str(lastMatchs)] = res.apply(lambda row: toBet(row), axis=1)
        if (i % 2 == 0) :
            data_res.at[data_res['Players'].str.contains(Player.replace("+", " ").strip()), 'Analysis1'] = round(df["Analysis"].mean(),2)
        else:
            data_res.at[data_res['Players'].str.contains(Player.replace("+", " ").strip()), 'Analysis2'] = round(df["Analysis"].mean(),2)
        
        res = data_res        
            
            ##Nb Win when favorite, Nb Loss when Favorite .....
            ##Ecart de cote for each case
            ## Nb sets win when loss favorite/not ... etc

        def toBet(row):
                # BUG ICI
            if (abs(row["Analysis1"] - row["Analysis2"])>threshold):
                if (row["Analysis1"] > row["Analysis2"]):
                    if (row["Odd1"] > row["Odd2"]):
                        return 10
                else:
                    if (row["Odd2"] > row["Odd1"]):
                        return 10
                return -10
            else:
                return 0
        res["toBet"] = res.apply(lambda row: toBet(row), axis=1)
    i=i+1

data_res.head(5)